In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed, LSTM, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Assuming you have your data loaded as X and y
# Example loading data (modify paths and variables as needed):
# X = np.load('your_data_X.npy')
# y = np.load('your_labels_y.npy')

# Define the parameters for the data shape
num_handsigns = 4  # Number of different hand signs
videos_per_handsign = 50
frames_per_video = 30
num_landmarks = 51
num_coordinates = 3

# Assuming X is already prepared (e.g., shape (num_handsigns * videos_per_handsign, frames_per_video, num_landmarks, num_coordinates))
# If not, load your data here

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a more lightweight and efficient model architecture for real-time processing
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(frames_per_video, num_landmarks, num_coordinates, 1)),
    TimeDistributed(BatchNormalization()),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Dropout(0.25)),
    
    TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
    TimeDistributed(BatchNormalization()),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Dropout(0.25)),
    
    TimeDistributed(Flatten()),
    
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_handsigns, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape X_train and X_val for the model
X_train = X_train.reshape(-1, frames_per_video, num_landmarks, num_coordinates, 1)
X_val = X_val.reshape(-1, frames_per_video, num_landmarks, num_coordinates, 1)

# Train the model with augmented data
history = model.fit(datagen.flow(X_train, y_train, batch_size=10), epochs=250, validation_data=(X_val, y_val))

# Save the updated model
model.save('optimized_handsigns_model.h5')

# Optionally, save the training history
import pickle
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Analysis of Training History
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# Save the plot if needed
# plt.savefig('training_history.png')
